In [ ]:
# =============================================================================
#
# Pronostico time trend + fourier seasonal
#
# =============================================================================

#
# Carga de datos
#
import nbimporter  #  type: ignore
import functions  #  type: ignore


df_orig = functions.load_data()
df_orig.head()

In [ ]:
#
# Componentes de tendencia lineal
#
df_orig = functions.add_linear_trend_component(df_orig)
df_orig = functions.add_month_component(df_orig)
df_orig.head()

In [ ]:
#
# Fourier
#
df_orig = functions.add_sin_cos_components(df_orig)
df_orig.head(15)

In [ ]:
#
# Particionamiento de los datos
#
(
    X_complete,
    y_complete,
    X_train,
    y_train,
    X_test,
    y_test,
) = functions.train_test_split(
    df=df_orig,
    x_columns=[
        "trend",
        "sin_12m",
        "cos_12m",
        "sin_6m",
        "cos_6m",
        "sin_4m",
        "cos_4m",
        "sin_3m",
        "cos_3m",
    ],
    y_column="yt_true",
)

In [ ]:
# Especificación del modelo
#
from sklearn.pipeline import Pipeline  #  type: ignore
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler  #  type: ignore
from sklearn.linear_model import LinearRegression  #  type: ignore
from sklearn.compose import ColumnTransformer  #  type: ignore

pipeline = Pipeline(
    [
        (
            "preprocessor",
            ColumnTransformer(
                [
                    (
                        "trend",
                        Pipeline(
                            [
                                ("poly", PolynomialFeatures(degree=3)),
                                ("scaler", MinMaxScaler()),
                            ]
                        ),
                        ["trend"],
                    ),
                ],
                remainder="passthrough",
            ),
        ),
        ("regressor", LinearRegression()),
    ]
)

In [ ]:
#
# Entrenamiento y pronostico
#
pipeline.fit(X_train, y_train)
df_orig["yt_pred_time_trend_3_fourier"] = pipeline.predict(X_complete)
functions.plot_time_series(df_orig)

In [ ]:
#
# Almacenamiento de los resultados
#
functions.save_forecasts(df_orig)

In [ ]:
#
# Métricas de error
#
metrics = functions.compute_evaluation_metrics(df_orig)
functions.save_metrics(metrics)
metrics